### Word In Visual Genome Process

In [7]:
#!pip install --upgrade pip

In [8]:
#!pip install --upgrade regex

In [9]:
#!pip install --upgrade pandas

In [ ]:
#!pip install --upgrade yellowbrick

In [3]:
#!pip install --upgrade scipy

In [10]:
#import re
#print(re.__version__)

In [11]:
#import pandas
#print(pandas.__version__)

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import re
import glob
from pathlib import Path
import shutil
from os.path import isfile, join

In [2]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

In [3]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Lemma Stem POS/Result/4-Word In Visual Genome"

Path(path).mkdir(parents=True, exist_ok=True)

In [39]:
def word_group_dataframe(df, search_list, target_column, sample_num):
    '''
    word_group_dataframe(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False)].sample(sample_num)
            #df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].tail(sample_num)  # will test
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

#### Visual Genome Data

In [5]:
df_genome_objects = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Result/3-Visual Genome Process/Visual_Genome_Objects_Analysis.csv")
df_genome_objects

,object_id,names,image_id,image_url,synsets,merged_object_ids,height,width,x_koor,y_koor
0,1058549,trees,1,https://cs.stanford.edu/people/rak248/VG_100K_...,tree.n.01,NaN,557,799,0,0
1,1058534,sidewalk,1,https://cs.stanford.edu/people/rak248/VG_100K_...,sidewalk.n.01,5046,290,722,78,308
2,1058508,building,1,https://cs.stanford.edu/people/rak248/VG_100K_...,building.n.01,NaN,538,222,1,0
3,1058539,street,1,https://cs.stanford.edu/people/rak248/VG_100K_...,street.n.01,3798578,258,359,439,283
4,1058543,wall,1,https://cs.stanford.edu/people/rak248/VG_100K_...,wall.n.01,NaN,535,135,0,1
...,...,...,...,...,...,...,...,...,...,...
2516934,3506171,number,2417997,http://crowdfile.blob.core.chinacloudapi.cn/46...,numeral.n.01,NaN,24,28,188,228
2516935,3786799,plate,2417997,http://crowdfile.blob.core.chinacloudapi.cn/46...,plate.n.04,NaN,54,89,238,233
2516936,3103226,road,2417997,http://crowdfile.blob.core.chinacloudapi.cn/46...,road.n.01,NaN,83,90,312,247
2516937,3094589,road,2417997,http://crowdfile.blob.core.chinacloudapi.cn/46...,road.n.01,NaN,121,495,2,208


In [6]:
df_genome_objects["names"].nunique()

82825

In [7]:
df_genome_attributes = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Result/3-Visual Genome Process/Visual_Genome_Attributes_Analysis.csv")
df_genome_attributes

,object_id,names,image_id,attributes,synsets,height,width,x_koor,y_koor
0,1058498,clock,1,"green , tall",clock.n.01,339,79,421,91
1,5046,street,1,sidewalk,street.n.01,262,714,77,328
2,5045,shade,1,NaN,shade.n.01,192,274,119,338
3,1058529,man,1,NaN,man.n.01,262,60,238,249
4,5048,sneakers,1,grey,gym_shoe.n.01,26,52,243,489
...,...,...,...,...,...,...,...,...,...
3802369,3422177,sky,2417997,"crystal clear , blue",sky.n.01,170,497,1,2
3802370,3103226,road,2417997,NaN,road.n.01,83,90,312,247
3802371,2712437,doors,2417997,NaN,door.n.01,144,50,98,127
3802372,3140197,bus,2417997,green,bus.n.01,155,225,29,109


In [8]:
df_genome_relationships = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Result/3-Visual Genome Process/Visual_Genome_Relationships_Analysis.csv")
df_genome_relationships

,image_id,obj_names,obj_object_id,obj_synsets,obj_merged_object_ids,obj_height,obj_width,obj_x_koor,obj_y_koor,relationships_id,synsets,sub_name,sub_object_id,sub_synsets,sub_height,sub_width,sub_x_koor,sub_y_koor,predicate
0,1,sidewalk,1058534,sidewalk.n.01,5046,290,722,78,308,15927,along.r.01,shade,5045,shade.n.01,192,274,119,338,on
1,1,shoes,1058525,shoe.n.01,5048,28,48,388,485,15928,wear.v.01,man,1058529,man.n.01,262,60,238,249,wears
2,1,NaN,5050,headlight.n.01,NaN,15,23,514,366,15929,have.v.01,car,5049,car.n.01,98,74,479,315,has
3,1,NaN,1058508,building.n.01,NaN,536,218,1,2,15930,along.r.01,sign,1058507,sign.n.02,182,88,118,13,on
4,1,NaN,1058534,sidewalk.n.01,NaN,266,722,77,331,15931,along.r.01,tree trunk,5055,trunk.n.01,327,87,622,234,on
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2316099,2417997,NaN,3103225,bus.n.01,NaN,193,289,14,110,4245033,along.r.01,plate,3786799,plate.n.04,54,89,238,233,on
2316100,2417997,NaN,3786800,desert.n.01,NaN,119,157,301,211,4245034,in.r.01,NaN,3103225,bus.n.01,182,287,14,110,in
2316101,2417997,NaN,3103226,road.n.01,NaN,83,90,312,247,3722727,along.r.01,bus,3103225,bus.n.01,193,289,14,110,on
2316102,2417997,NaN,2712437,door.n.01,NaN,144,50,98,127,3880229,NaN,bus,3103225,bus.n.01,193,289,14,110,with


#### Word Search In Genome Data

In [9]:
df_word_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Result/2-Word Select Process/{lang_folder.capitalize()}_45000_Lemma_Pos_Process.xlsx")
df_word_select

,pos.ud_imst,word,frequency,lemma.spacylookup,stem,word_en_translate,lemma_en_translate
0,NUM,bir,18835735,bir,NaN,a,a
1,DET,bu,11062659,bu,NaN,this,this
2,PRON,ne,8025880,ne,NaN,what,what
3,CCONJ,ve,7766036,ve,NaN,and,and
4,ADP,için,5484109,için,NaN,for,for
...,...,...,...,...,...,...,...
44995,NOUN,hayırbu,871,hayırbu,NaN,no,no
44996,NOUN,konferansına,871,konferans,NaN,to the conference,conference
44997,NOUN,kardeşisin,871,kardeş,NaN,your brother,brother
44998,VERB,kaldırıyorsun,871,kaldır,NaN,you remove,lift


In [10]:
word_list = ['an','anlama','doğum','gidiyor','günün','istediğim','istediğin','istediğiniz','kutlu','olmuş','soru','söylemek','zamanki','zor','şuna']

In [11]:
df_word_raw = df_word_select[df_word_select["word"].isin(word_list)]
df_word_raw.reset_index(drop=True, inplace=True)
df_word_raw.drop(["frequency","stem"], inplace=True, axis=1)
df_word_raw 

<ipython-input-11-edd78fc4eb41>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_raw.drop(["frequency","stem"], inplace=True, axis=1)


,pos.ud_imst,word,lemma.spacylookup,word_en_translate,lemma_en_translate
0,NOUN,an,an,moment,moment
1,ADJ,zor,zor,hard,hard
2,VERB,gidiyor,git,going,go
3,VERB,olmuş,ol,dead,die
4,NOUN,istediğim,iste,what i want,at work
5,VERB,söylemek,söyle,to say,say
6,NOUN,şuna,şu,to this,this
7,NOUN,soru,soru,question,question
8,NOUN,doğum,doğum,birth,birth
9,NOUN,istediğin,iste,you want,at work


In [12]:
df_word_raw_lemma_trans_list = df_word_raw["lemma_en_translate"].to_list()
df_word_raw_word_trans_list = df_word_raw["word_en_translate"].to_list()

In [40]:
df_genome_search_result = word_group_dataframe(df_genome_objects, df_word_raw_lemma_trans_list, "names", 20)
df_genome_search_result.rename(columns={"search_string":"lemma_en_translate"}, inplace=True)
df_genome_search_result

,lemma_en_translate,object_id,names,image_id,image_url,synsets,merged_object_ids,height,width,x_koor,y_koor
0,hard,4122046,hard drive,2328198,http://crowdfile.blob.core.chinacloudapi.cn/46...,NaN,NaN,92,102,249,125
1,hard,2991976,hard hat,2373969,http://crowdfile.blob.core.chinacloudapi.cn/46...,hat.n.01,"2319184 , 1752048 , 2312864",97,86,97,50
2,hard,1642715,hard hat,1667,http://crowdfile.blob.core.chinacloudapi.cn/46...,hat.n.01,NaN,127,142,0,377
3,hard,1757980,hard hat,2347159,http://crowdfile.blob.core.chinacloudapi.cn/46...,hat.n.01,NaN,5,6,252,212
4,hard,475514,hard drive,2393025,http://crowdfile.blob.core.chinacloudapi.cn/46...,drive.n.01,NaN,48,50,116,115
...,...,...,...,...,...,...,...,...,...,...,...
128,time,4668871,time,2333725,http://crowdfile.blob.core.chinacloudapi.cn/46...,NaN,4668872,9,13,72,114
129,time,3882533,time,2365776,http://crowdfile.blob.core.chinacloudapi.cn/46...,clock_time.n.01,NaN,20,22,423,137
130,time,607474,time,2369979,http://crowdfile.blob.core.chinacloudapi.cn/46...,clock_time.n.01,NaN,41,128,150,155
131,time,226435,time,2410081,http://crowdfile.blob.core.chinacloudapi.cn/46...,clock_time.n.01,NaN,12,9,265,127


In [28]:
df_genome_search_result["lemma_en_translate"].nunique()

10

In [29]:
df_genome_search_result_merge = pd.merge(df_genome_search_result,df_word_raw,how="left",on="lemma_en_translate")
df_genome_search_result_merge

,lemma_en_translate,object_id,names,image_id,image_url,synsets,merged_object_ids,height,width,x_koor,y_koor,pos.ud_imst,word,lemma.spacylookup,word_en_translate
0,hard,1805716,hard terrain,2374891,http://crowdfile.blob.core.chinacloudapi.cn/46...,terrain.n.01,NaN,297,207,288,71,ADJ,zor,zor,hard
1,hard,1757981,hard hat,2347159,http://crowdfile.blob.core.chinacloudapi.cn/46...,hat.n.01,1908324,6,6,222,211,ADJ,zor,zor,hard
2,hard,1054297,hard drive,2316366,http://crowdfile.blob.core.chinacloudapi.cn/46...,drive.n.01,NaN,56,79,228,279,ADJ,zor,zor,hard
3,hard,540130,hard hats,2382266,http://crowdfile.blob.core.chinacloudapi.cn/46...,hat.n.01,NaN,8,27,64,168,ADJ,zor,zor,hard
4,hard,1820994,hard paper,2346372,http://crowdfile.blob.core.chinacloudapi.cn/46...,paper.n.01,NaN,64,438,62,315,ADJ,zor,zor,hard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,time,4165471,day time,2392324,http://crowdfile.blob.core.chinacloudapi.cn/46...,NaN,NaN,332,500,0,0,NOUN,zamanki,zaman,the time
129,time,769622,time,2362969,http://crowdfile.blob.core.chinacloudapi.cn/46...,clock_time.n.01,NaN,16,80,262,330,NOUN,zamanki,zaman,the time
130,time,1201648,time,2395978,http://crowdfile.blob.core.chinacloudapi.cn/46...,clock_time.n.01,NaN,21,23,183,237,NOUN,zamanki,zaman,the time
131,time,2423625,time 4:49,2347350,http://crowdfile.blob.core.chinacloudapi.cn/46...,clock_time.n.01,NaN,154,138,239,128,NOUN,zamanki,zaman,the time


In [30]:
#df_genome_search_result_merge.to_excel("test1.xlsx", index=False)

In [31]:
df_genome_search_result2 = word_group_dataframe(df_genome_objects, df_word_raw_word_trans_list, "names", 20)
df_genome_search_result2.rename(columns={"search_string":"word_en_translate"}, inplace=True)
df_genome_search_result2

,word_en_translate,object_id,names,image_id,image_url,synsets,merged_object_ids,height,width,x_koor,y_koor
0,hard,2148534,hard hat,2349708,http://crowdfile.blob.core.chinacloudapi.cn/46...,hat.n.01,1683116,21,17,194,145
1,hard,3898907,hard hat,2351040,http://crowdfile.blob.core.chinacloudapi.cn/46...,NaN,NaN,33,32,209,84
2,hard,2798621,hard hat,2336789,http://crowdfile.blob.core.chinacloudapi.cn/46...,hat.n.01,NaN,37,59,253,65
3,hard,1746578,hard hat,2370647,http://crowdfile.blob.core.chinacloudapi.cn/46...,hat.n.01,NaN,160,258,173,0
4,hard,980857,hard,2327221,http://crowdfile.blob.core.chinacloudapi.cn/46...,NaN,NaN,30,152,38,170
...,...,...,...,...,...,...,...,...,...,...,...
68,question,1001753,question mark,2319465,http://crowdfile.blob.core.chinacloudapi.cn/46...,question_mark.n.01,NaN,12,15,466,228
69,question,1054852,question mark,2316283,http://crowdfile.blob.core.chinacloudapi.cn/46...,question_mark.n.01,NaN,10,5,211,190
70,birth,272967,birth,2407824,http://crowdfile.blob.core.chinacloudapi.cn/46...,birth.n.01,NaN,165,492,7,86
71,birth,4101782,birth date,2405799,http://crowdfile.blob.core.chinacloudapi.cn/46...,NaN,NaN,18,74,112,68


In [32]:
df_genome_search_result2["word_en_translate"].nunique()

5

In [33]:
df_genome_search_result_merge2 = pd.merge(df_genome_search_result2,df_word_raw,how="left",on="word_en_translate")
df_genome_search_result_merge2

,word_en_translate,object_id,names,image_id,image_url,synsets,merged_object_ids,height,width,x_koor,y_koor,pos.ud_imst,word,lemma.spacylookup,lemma_en_translate
0,hard,2148534,hard hat,2349708,http://crowdfile.blob.core.chinacloudapi.cn/46...,hat.n.01,1683116,21,17,194,145,ADJ,zor,zor,hard
1,hard,3898907,hard hat,2351040,http://crowdfile.blob.core.chinacloudapi.cn/46...,NaN,NaN,33,32,209,84,ADJ,zor,zor,hard
2,hard,2798621,hard hat,2336789,http://crowdfile.blob.core.chinacloudapi.cn/46...,hat.n.01,NaN,37,59,253,65,ADJ,zor,zor,hard
3,hard,1746578,hard hat,2370647,http://crowdfile.blob.core.chinacloudapi.cn/46...,hat.n.01,NaN,160,258,173,0,ADJ,zor,zor,hard
4,hard,980857,hard,2327221,http://crowdfile.blob.core.chinacloudapi.cn/46...,NaN,NaN,30,152,38,170,ADJ,zor,zor,hard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,question,1001753,question mark,2319465,http://crowdfile.blob.core.chinacloudapi.cn/46...,question_mark.n.01,NaN,12,15,466,228,NOUN,soru,soru,question
69,question,1054852,question mark,2316283,http://crowdfile.blob.core.chinacloudapi.cn/46...,question_mark.n.01,NaN,10,5,211,190,NOUN,soru,soru,question
70,birth,272967,birth,2407824,http://crowdfile.blob.core.chinacloudapi.cn/46...,birth.n.01,NaN,165,492,7,86,NOUN,doğum,doğum,birth
71,birth,4101782,birth date,2405799,http://crowdfile.blob.core.chinacloudapi.cn/46...,NaN,NaN,18,74,112,68,NOUN,doğum,doğum,birth


In [37]:
#df_genome_search_result_merge2.to_excel("test2.xlsx", index=False)

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_45000_Lemma_Pos_Process.xlsx")
output_file

In [ ]:
for l in output_file:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file:
    try:
        os.remove(j)
    except:
        pass